In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# %pip install --upgrade -q pip
# %pip install -q playwright
# !playwright install

In [3]:
# from playwright.sync_api import sync_playwright #NOT working in Jupyter notebooks
# with sync_playwright() as p:
#     browser = p.chromium.launch(headless=False)
#     page = browser.new_page()
#     # Your automation code here
#     browser.close()

In [4]:
import asyncio
from playwright.async_api import async_playwright, expect

In [5]:
import requests

# Playwright list
- Browser > Context > Page
- Visit url
- find by text
- find by css
- find by xpath
- find multiple elements and count them
- find inside an element
- visit parent
- visit ancestor
- Reload page
- get text
- clicking
- typing
- mouse hover
- right clicking with context menu
- double clicking
- drag n drop
- pressing tab key
- pressing enter key
- submitting a form
- selecting native dropdown
- getting attribute
- setting attribute
- dismiss alert
- confirm alert press ok
- confirm alert press cancel
- insert for prompt alert
- visit an iframe
- visit a nested iframe
- open new tab
- visit new tab
- open new window
- visit new window  
- upload a file
- download a file
- set browser geolocation
- Go back/forward on pages
- Cookies

============

- return to main frame <--- Selenium only
- return to main window <--- Selenium only
- visit original tab <-- Selenium only

# Configuring Viewport Size and Global Timeout

In [6]:
pw = await async_playwright().start()
browser = await pw.chromium.launch(headless=False)
# page = await browser.new_page() # better exploit context than launching a new page directly
context = await browser.new_context(
    viewport={'width': 1920, 'height': 1080},
)

# Setting implicit global timeout
context.set_default_timeout(5000)

page = await context.new_page()

# I visit a url

In [7]:
await page.goto('https://qaplayground.dev/apps/dynamic-table/')

<Response url='https://qaplayground.dev/apps/dynamic-table/' request=<Request url='https://qaplayground.dev/apps/dynamic-table/' method='GET'>>

# I find elements by text and get their text
Also compare innerText with text_content

In [8]:
loc = page.get_by_text(text='superhero', exact=False)
inner_text = await loc.inner_text() #how it is rendered on the website
txt = await loc.text_content() #content from inside the html
assert txt != inner_text
column_name = await page.get_by_text(text='superhero', exact=False).inner_text()
assert column_name == 'SUPERHERO'

# I find elements by CSS selector

In [9]:
top_title = page.locator("a[aria-label='full courses']")
await expect(top_title).to_have_text('Apps')

# I find elements by XPath selector

In [10]:
top_right_title = page.locator("//*[contains(text(), 'View Test Suite')]")
await expect(top_right_title).to_be_visible()

# I find multiple elements and count them

In [11]:
elems = page.locator('tr > td')
assert await elems.count() == 24
await expect(elems).to_have_count(24)

# I find an element inside an element

In [12]:
first_row = page.locator('thead > tr:nth-of-type(1)')
assert await first_row.count() == 1
second_header = first_row.locator('th:nth-of-type(2)')
await expect(second_header).to_contain_text('Status')

# I get parent of element

In [13]:
superhero = page.get_by_text('Spider-Man', exact=True)
assert await superhero.is_visible()
parent = superhero.locator('..') #using xpath
email = parent.get_by_text('@avengers.com')
await expect(email).to_have_text('spider-man@avengers.com')

# I get ancestor of element

In [14]:
spiderman = page.get_by_text('Spider-Man', exact=True)
spiderman_container = spiderman.locator('//ancestor::tr')
spiderman_realname_col = spiderman_container.locator('td:nth-of-type(3)')
assert await spiderman_realname_col.count() == 1
assert 'Peter Parker' in await spiderman_realname_col.text_content()

# I reload the page

In [15]:
await page.reload()

<Response url='https://qaplayground.dev/apps/dynamic-table/' request=<Request url='https://qaplayground.dev/apps/dynamic-table/' method='GET'>>

# I can click and I can type

In [16]:
import re
await page.goto('https://qaplayground.dev/apps/verify-account/')
await expect(page.locator('#title')).to_have_text('Verify Your Account')

nums = [page.locator(f'.code-container input:nth-of-type({num})') for num in range(1, 6+1)]
assert len(nums) == 6

code_regex = '-'.join(['\d'] * 6)
code_pattern = re.compile(code_regex)
assert code_pattern.search(' The confirmation code is 9-9-9-9-9-9 ').group() == '9-9-9-9-9-9'
phrase = await page.locator('.info').inner_text()

code_nums = code_pattern.search(phrase).group().split('-')
assert len(code_nums) == 6

for ii in range(6):
  await nums[ii].click()
  await nums[ii].type(code_nums[ii])

await expect(page.locator('.info.success')).to_have_text('Success')

# I can execute a real hover with the mouse

In [17]:
await page.goto('https://qaplayground.dev/apps/mouse-hover/')

await expect(page.locator('.poster')).to_be_visible()

# async def get_css_property(selector, property_name):
#     # Combine parameters into a single object and destructure them inside the JavaScript function
#     return await page.evaluate('''({ selector, property_name }) => {
#         const element = document.querySelector(selector);
#         return window.getComputedStyle(element).getPropertyValue(property_name);
#         }''', { 'selector': selector, 'property_name': property_name })

target_elem = '.title-container'
target_css_property = 'opacity'

# await get_css_property(selector = target_elem, property_name = target_css_property)

await expect(page.locator(target_elem)).to_have_css('opacity', '0')

await page.locator('.poster').hover()

await expect(page.locator(target_elem)).to_have_css('opacity', '1')

# I can right click and interact with a context menu

In [18]:
await page.goto('https://qaplayground.dev/apps/context-menu/')

assert not await page.locator('i.uil-trash-alt').is_visible()

await page.locator('#msg').click(button='right')

assert await page.locator('i.uil-trash-alt').is_visible()

# I can double click and the action to be recorded as so

In [19]:
await page.goto('https://play1.automationcamp.ir/mouse_events.html')
assert not await page.locator('#click_type').is_visible()
await page.locator('#click_area').click(click_count=2)
assert await page.locator('#click_type').is_visible()
assert await page.locator('#click_type').text_content() == 'Double-Click'

# I drag n drop an element on another element

In [20]:
drop_elem = page.locator('#drop_target')
drag_elem = page.locator('#drag_source')

await expect(drop_elem).to_be_visible()
await expect(drop_elem).to_have_text('Target')

await expect(drag_elem).to_have_text('Drop me on to the green box')

await page.drag_and_drop('#drag_source', '#drop_target')

await expect(drop_elem).to_have_text('Drop is successful!')

# I use tab key and enter key to fill and submit a form

In [21]:
await page.goto('https://play1.automationcamp.ir/forms.html')

await expect(page.locator('#validationCustom04')).not_to_be_focused()
await page.focus('#validationCustom03')
await page.locator('#validationCustom03').type('beautiful city')

await page.press('#validationCustom03', 'Tab')
await expect(page.locator('#validationCustom04')).to_be_focused()
await page.locator('#validationCustom04').type('MaState')

await expect(page.locator('#validationCustom05')).not_to_be_focused()
await page.press('#validationCustom04', 'Tab')
await expect(page.locator('#validationCustom05')).to_be_focused()

loc = page.locator('#invalid_terms')
await expect(loc).to_contain_text('You must agree before submitting')
await expect(loc).not_to_be_visible()

await page.locator('#validationCustom05').type('53479')
await page.press('#validationCustom05', 'Enter')

await expect(loc).to_be_visible()

# I submit a form using native browser submit function

In [22]:
form_loc = 'form.needs-validation'
await page.locator(f'{form_loc} .form-check-input').check()
await expect(page.locator('#validationCustom03')).to_have_value('beautiful city')
await page.evaluate(f'document.querySelector("{form_loc}").submit()')
await expect(page.locator('#validationCustom03')).to_have_value('')

# I select a native dropdown from a form

In [23]:
loc = page.locator('#select_tool')
assertion_loc = page.locator('#select_tool_validate')
assert await loc.is_visible()
assert await assertion_loc.is_hidden()
await expect(assertion_loc).to_have_text('')
await loc.select_option('cyp')
assert await assertion_loc.is_visible()
await expect(assertion_loc).to_contain_text('cyp')

# I get and I set attributes from an html element

In [24]:
css_sel = '#check_java'
loc = page.locator(css_sel)
await expect(loc).to_have_attribute('disabled', '')
await page.evaluate(f'document.querySelector("{css_sel}").removeAttribute("disabled")')
await expect(loc).not_to_have_attribute('disabled', '')
await page.evaluate(f'document.querySelector("{css_sel}").setAttribute("disabled", "true")')
await expect(loc).to_have_attribute('disabled', 'true')

# I wait for the contents of a popup alert and I dismiss it

In [25]:
await page.goto('https://play1.automationcamp.ir/expected_conditions.html')
sel = 'button#alert_trigger'
alert_msg = 'I am alerting you!'

assert await page.locator(sel).text_content() == 'Show Alert'

assert not await page.locator('#alert_handled_badge').is_visible()

async with page.expect_event('dialog') as dialog_info:
  await page.click(sel)
  
dialog = await dialog_info.value
assert dialog.type == 'alert'
assert dialog.message == alert_msg
await dialog.accept()

await page.locator('#alert_handled_badge').wait_for(state='visible')
alert_badge = page.locator('#alert_handled_badge')
assert 'Alert' in await alert_badge.text_content()
assert 'handled' in await alert_badge.text_content()

# I wait for the confirmation popup and I accept it

In [26]:
sel = 'button#prompt_trigger'
confirmation_msg = "Choose wisely...\nIt's your life!"
confirm_badge = '#confirm_ok_badge'

await expect(page.locator(sel)).to_have_text('Show Prompt')

assert not await page.locator(confirm_badge).is_visible()

async with page.expect_event('dialog') as dialog_info: # we have prepared the listener but we are not blocking execution
  await page.click(sel)

dialog = await dialog_info.value
assert dialog.type == 'confirm'
assert dialog.message == confirmation_msg
await dialog.accept()

await expect(page.locator(confirm_badge)).to_be_visible()
await expect(page.locator(confirm_badge)).to_contain_text('OK')

# I wait for the confirmation popup and I reject it

In [27]:
sel = 'button#prompt_trigger'
confirmation_msg = "Choose wisely...\nIt's your life!"
confirm_badge = '#confirm_cancelled_badge'

await page.reload()

await expect(page.locator(sel)).to_have_text('Show Prompt')

assert not await page.locator(confirm_badge).is_visible()

async with page.expect_event('dialog') as dialog_info: # we have prepared the listener but we are not blocking execution
  await page.click(sel)

dialog = await dialog_info.value
assert dialog.type == 'confirm'
assert dialog.message == confirmation_msg
await dialog.dismiss()

await expect(page.locator(confirm_badge)).to_be_visible()
await expect(page.locator(confirm_badge)).to_contain_text('Cancelled')

# I trigger a prompt alert and pass "geia sou" as text TODO also did not work for Playwright
WHICH MEANS THAT WE NEED TO FIND ANOTHER WEBSITE TO TEST THIS as it might have to do with how people are getting their info

In [28]:
# await page.goto('https://testpages.herokuapp.com/styled/alerts/alert-test.html')
# txt = 'geia sou'
# html_sel = '#promptreturn'
# button = '#promptexample'
# prompt_msg = 'I prompt you'
# await expect(page.locator(html_sel)).to_have_text('')
# async with page.expect_event('dialog') as dialog_info: # we have prepared the listener but we are not blocking execution
#   await page.click(button)

# dialog = await dialog_info.value
# assert dialog.type == 'prompt'
# assert dialog.message == prompt_msg
# await dialog.accept(txt)

# I can assert inside a changing iframe

In [29]:
await page.goto('https://qaplayground.dev/apps/changing-iframe/')
frame_element = await page.wait_for_selector('iframe')
frame = await frame_element.content_frame()
elem = await frame.wait_for_selector('#msg', timeout=20000)
await expect(frame.locator('#msg')).to_have_text('This is the end of the journey')

# I can access a nested iframe and assert in it

In [30]:
await page.goto('https://qaplayground.dev/apps/iframe/')

await expect(page.locator('#frame1')).to_be_visible()

frame_elem = await page.wait_for_selector('#frame1')
frame = await frame_elem.content_frame()

await expect(frame.locator('legend')).to_have_text('First Level Iframe')
await expect(frame.locator('#frame2')).to_be_visible()

frame2_elem = await frame.wait_for_selector('#frame2')
inner_frame = await frame2_elem.content_frame()

await expect(inner_frame.locator('legend')).to_have_text('Second Level Iframe')

await expect(inner_frame.locator('#msg')).not_to_be_visible()
await expect(inner_frame.locator('.btn')).to_be_visible()

await inner_frame.locator('.btn').click()

await expect(inner_frame.locator('#msg')).to_be_visible()
await expect(inner_frame.locator('#msg')).to_have_text('Button Clicked')


# I can manage new tabs

In [31]:
await page.goto('https://qaplayground.dev/apps/new-tab/')
async with context.expect_page() as new_page_info:
  await page.click('#open')
new_page = await new_page_info.value
await new_page.wait_for_load_state()
await expect(new_page.locator('h1')).to_be_visible()
await expect(new_page.locator('h1')).to_have_text('Welcome to the new page!')
await new_page.close()

# I can manage new windows

In [32]:
await page.goto('https://qaplayground.dev/apps/popup/')

await expect(page.locator('#login')).to_have_text('Open')

async with context.expect_page() as popup_info:
  await page.locator('#login').click()

popup = await popup_info.value
await popup.wait_for_load_state()

await expect(popup.locator('button')).to_have_text('Submit')

assert popup in context.pages

await popup.locator('button').click()

assert popup not in context.pages

# I can upload a file

In [33]:
# import os
# notebooks = [file for file in os.listdir() if file.endswith('.ipynb')]
# assert len(notebooks) == 1
# notebook = notebooks[0]
# notebook

In [34]:
await page.goto('https://qaplayground.dev/apps/upload/')
await expect(page.locator('input[type="file"]')).to_be_attached()
myfile = 'balls.jpg'
await page.set_input_files('input[type="file"]', myfile) #actually uploading the file
await expect(page.locator('#num-of-files')).to_have_text('1 File Selected')
await expect(page.locator('figcaption')).to_have_text(myfile)

# I can download a file

In [35]:
await page.goto('https://qaplayground.dev/apps/download/')
await expect(page.locator('#file')).to_have_text('Download ⏬')
async with page.expect_download() as download_info:
    await page.click('#file')

download = await download_info.value
path = await download.path()
await download.save_as('sample.pdf')

def is_pdf(file_path):
    try:
        with open(file_path, 'rb') as file:
            header = file.read(5)
            return header == b'%PDF-'
    except Exception as e:
        print(f"An error occurred: {e}")
        return False

assert is_pdf('sample.pdf')

# I can set the Geolocation to custom coordinates

In [36]:
await context.close()
await browser.close()

browser = await pw.chromium.launch(headless=False)
context = await browser.new_context(
    geolocation={"latitude": 37.7749, "longitude": -122.4194},
    permissions=["geolocation"]
)
page = await context.new_page()
await page.goto('https://qaplayground.dev/apps/geolocation/')

await expect(page.locator('#location-info')).to_have_text('Click on the button to get your current location')
await page.click('#get-location')
await expect(page.locator('#location-info')).to_have_text('San Francisco, United States')

# I can move forward and backward to pages

In [37]:
await page.goto('https://qaplayground.dev/apps/links/')

await expect(page.locator('xpath=//* >> text="Home"')).to_be_visible()
await expect(page.locator('xpath=//* >> text="Contact"')).to_be_visible()
await expect(page.locator('#title')).to_have_count(0)

await page.click('div#nav > a:nth-of-type(5)')

await expect(page.locator('#title')).to_have_text('Welcome to the Contact Page')
await expect(page.locator('xpath=//* >> text="Contact"')).not_to_be_attached()

await page.go_back()

await expect(page.locator('xpath=//* >> text="Home"')).to_be_visible()
await expect(page.locator('xpath=//* >> text="Contact"')).to_be_visible()
await expect(page.locator('#title')).to_have_count(0)

await page.go_forward()

await expect(page.locator('#title')).to_have_text('Welcome to the Contact Page')
await expect(page.locator('xpath=//* >> text="Contact"')).not_to_be_attached()

# I can get, set, clear and save Cookies of a webpage

In [38]:
import json

In [39]:
await page.goto('https://www.saucedemo.com/')
await page.fill('#user-name', 'standard_user')
await page.fill('#password', 'secret_sauce')
await page.click('#login-button')
await page.wait_for_url('https://www.saucedemo.com/inventory.html')
assert page.url == 'https://www.saucedemo.com/inventory.html'
title = await page.locator('.title').inner_text()
assert title == 'Products'
cookies = await context.cookies() # save cookies
with open('cookies.json', 'w') as f:
    json.dump(cookies, f)
await context.clear_cookies()
await page.reload()
await page.wait_for_url('https://www.saucedemo.com/')
assert page.url == 'https://www.saucedemo.com/'
with open('cookies.json', 'r') as f:
    cookies = json.load(f)
await context.add_cookies(cookies) #load cookies
await page.goto('https://www.saucedemo.com/inventory.html')
await page.wait_for_url('https://www.saucedemo.com/inventory.html')
assert page.url == 'https://www.saucedemo.com/inventory.html'
title = await page.locator('.title').inner_text()
assert title == 'Products'

# END

In [40]:
# Close the browser and Playwright
await browser.close()
await pw.stop()

- Playwright has built-in parallelization support, whereas Selenium requires a third-party tool.
- Playwright executes faster than Selenium.
- Selenium doesn't support features like detailed reporting and video recording, while Playwright provides built-in support.

====

- Selenium can be used in real devices and remote servers, while Playwright doesn't offer this option.
- Selenium supports more browsers and programming languages than its opponent.